In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
training_descriptions = pd.read_csv('Training_set.csv')

In [3]:
training_descriptions

,filename,label
0,Image_1.jpg,SOUTHERN DOGFACE
1,Image_2.jpg,ADONIS
2,Image_3.jpg,BROWN SIPROETA
3,Image_4.jpg,MONARCH
4,Image_5.jpg,GREEN CELLED CATTLEHEART
...,...,...
6494,Image_6495.jpg,MANGROVE SKIPPER
6495,Image_6496.jpg,MOURNING CLOAK
6496,Image_6497.jpg,APPOLLO
6497,Image_6498.jpg,ELBOWED PIERROT


In [8]:
test1 = np.array(Image.open('Train/Image_1.jpg'))
test2 = np.array(Image.open('Train/Image_2.jpg'))

test.shape

(224, 224, 3)

In [14]:
combined = np.stack((test1, test2), axis = 0)

combined.shape

(2, 224, 224, 3)

In [18]:
training_descriptions['filename']

0          Image_1.jpg
1          Image_2.jpg
2          Image_3.jpg
3          Image_4.jpg
4          Image_5.jpg
             ...      
6494    Image_6495.jpg
6495    Image_6496.jpg
6496    Image_6497.jpg
6497    Image_6498.jpg
6498    Image_6499.jpg
Name: filename, Length: 6499, dtype: object

In [23]:
# initialize image data
training_data = np.array(Image.open('Train/Image_1.jpg'))

In [25]:
for i in range(1, len(training_descriptions)):

    print(training_data.shape)

    if i > 1:
        image_values = np.array(Image.open('Train/' + training_descriptions['filename'][i])).reshape((1, 224, 224, 3))
    else:   
        image_values = np.array(Image.open('Train/' + training_descriptions['filename'][i]))
        
    print(image_values.shape)

    training_data = np.stack((training_data, image_values), axis = 0)

(224, 224, 3)
(224, 224, 3)
(2, 224, 224, 3)
(1, 224, 224, 3)


ValueError: all input arrays must have the same shape

In [26]:
x = np.array([np.array(Image.open('Train/' + image_jpg)) for image_jpg in training_descriptions['filename']])

In [27]:
x.shape

(6499, 224, 224, 3)